In [1]:
import torch
import torch.nn as nn
from torchvision import transforms
import torch.nn.functional as F
from PIL import Image
import numpy as np

In [2]:
labels = ["birad-0","birad-1","birad-2","birad-3","birad-4","birad-5", "density-a","density-b","density-c","density-d"]

In [3]:
class MultiClassifier(nn.Module):
    def __init__(self):
        super(MultiClassifier, self).__init__()
        self.ConvLayer1 = nn.Sequential(
            nn.Conv2d(3, 64, 3), # 3, 256, 256
            nn.MaxPool2d(2), # op: 16, 127, 127
            nn.ReLU(), # op: 64, 127, 127
        )
        self.ConvLayer2 = nn.Sequential(
            nn.Conv2d(64, 128, 3), # 64, 127, 127   
            nn.MaxPool2d(2), #op: 128, 63, 63
            nn.ReLU() # op: 128, 63, 63
        )
        self.ConvLayer3 = nn.Sequential(
            nn.Conv2d(128, 256, 3), # 128, 63, 63
            nn.MaxPool2d(2), #op: 256, 30, 30
            nn.ReLU() #op: 256, 30, 30
        )
        self.ConvLayer4 = nn.Sequential(
            nn.Conv2d(256, 512, 3), # 256, 30, 30
            nn.MaxPool2d(2), #op: 512, 14, 14
            nn.ReLU(), #op: 512, 14, 14
            nn.Dropout(0.2)
        )
        self.Linear1 = nn.Linear(512 * 14 * 14, 1024)
        self.Linear2 = nn.Linear(1024, 256)
        self.Linear3 = nn.Linear(256, 10)
        
        
    def forward(self, x):
        x = self.ConvLayer1(x)
        x = self.ConvLayer2(x)
        x = self.ConvLayer3(x)
        x = self.ConvLayer4(x)
        x = x.view(x.size(0), -1)
        x = self.Linear1(x)
        x = self.Linear2(x)
        x = self.Linear3(x)
        return F.sigmoid(x)



In [4]:
model = torch.load("/home/single1/BACKUP/SamHUyen/mammo/sam/singleview_sam/model/model_epoch.pt", map_location=torch.device('cpu'))
model = model.eval()

In [5]:
def get_tensor(img):
    tfms = transforms.Compose([
        transforms.Resize((256, 256)),
        transforms.ToTensor()
    ])
    return tfms(Image.open(img).convert('RGB')).unsqueeze(0)

In [6]:
def predict(img, label_lst, model):
    tnsr = get_tensor(img)
    op = model(tnsr)
    op_b = torch.round(op)
    
    op_b_np = torch.Tensor.cpu(op_b).detach().numpy()
    
    preds = np.where(op_b_np == 1)[1]
    
    sigs_op = torch.Tensor.cpu(torch.round((op)*100)).detach().numpy()[0]
    
    o_p = np.argsort(torch.Tensor.cpu(op).detach().numpy())[0][::-1]
    
#     print("Argsort: {}".format(o_p))
#     print("Softmax: {}".format(sigs_op))
    
    
    #print(op_b_np)
    #print(preds)
    """
    label = []
    for i in preds:
        label.append(label_lst[i])
        
    arg_s = {}
    for i in o_p:
#         arg_s.append(label_lst[int(i)])
        arg_s[label_lst[int(i)]] = sigs_op[int(i)]
    """ 
    return op_b_np#label, list(arg_s.items())[:10]

In [14]:
predict('/home/single1/BACKUP/SamHUyen/multi_view_mammo_classification/images/1.2.410.200010.1064739.6393.1124429.1139296.1139296/1.3.12.2.1107.5.12.7.5054.30000020051200135573400000007.png', labels, model)

[[0. 1. 0. 0. 0. 0. 0. 0. 1. 0.]]
[1 8]
/home/single1/.local/lib/python3.8/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")


(['birad-1', 'density-c'],
 [('density-c', 97.0),
  ('birad-1', 68.0),
  ('birad-2', 27.0),
  ('birad-3', 3.0),
  ('birad-4', 2.0),
  ('density-b', 2.0),
  ('density-d', 1.0),
  ('birad-5', 0.0),
  ('birad-0', 0.0),
  ('density-a', 0.0)])

In [7]:
import pandas as pd
labels_df = pd.read_csv("/home/single1/BACKUP/SamHUyen/mammo/huyen/csv_singleview.csv")
labels_df["path"] = "/home/single1/BACKUP/SamHUyen/multi_view_mammo_classification/images/"+ labels_df["study_id"] + "/"  + labels_df["image_id"] + ".png" 
list_holdout = labels_df[labels_df["fold"]=="holdout"]["path"].values


In [8]:
from tqdm import tqdm
label_holdout = []
for img in tqdm(list_holdout):
    label_holdout.append(predict(img, labels, model))

  0%|          | 0/2900 [00:00<?, ?it/s]/home/single1/.local/lib/python3.8/site-packages/torch/nn/functional.py:1639: UserWarning: nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.
  warnings.warn("nn.functional.sigmoid is deprecated. Use torch.sigmoid instead.")
100%|██████████| 2900/2900 [05:36<00:00,  8.63it/s]


In [9]:
len(label_holdout)

2900

In [10]:
label_holdout[:3]

[array([[0., 1., 0., 0., 0., 0., 0., 0., 1., 0.]], dtype=float32),
 array([[0., 1., 0., 0., 0., 0., 0., 0., 1., 0.]], dtype=float32),
 array([[0., 1., 0., 0., 0., 0., 0., 0., 1., 0.]], dtype=float32)]

In [22]:
labels_df["predict"] = label_holdout
labels_df.head(5)

,patient_id,study_id,series_id,image_id,image_height,image_width,bbox,blabel,bsession_id,gbirad,gdensity,gscope,glaterality,gview_position,gsession_id,multilabel,fold,path,predict
0,2005012602,1.2.410.200010.1064800.1782.1119751.1134541.11...,1.3.12.2.1107.5.12.7.5054.30000020050900111600...,1.3.12.2.1107.5.12.7.5054.30000020050900111600...,3518,2800,NaN,No Box,NaN,BI-RADS 1,DENSITY-C,NaN,R,CC,NaN,"[0, 1, 0, 0, 0, 0, 0, 0, 1, 0]",train,/home/single1/BACKUP/SamHUyen/multi_view_mammo...,"[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,..."
1,2005012602,1.2.410.200010.1064800.1782.1119751.1134541.11...,1.3.12.2.1107.5.12.7.5054.30000020050900111600...,1.3.12.2.1107.5.12.7.5054.30000020050900111600...,3518,2800,NaN,No Box,NaN,BI-RADS 1,DENSITY-C,NaN,L,CC,NaN,"[0, 1, 0, 0, 0, 0, 0, 0, 1, 0]",train,/home/single1/BACKUP/SamHUyen/multi_view_mammo...,"[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,..."
2,2005012602,1.2.410.200010.1064800.1782.1119751.1134541.11...,1.3.12.2.1107.5.12.7.5054.30000020050900111600...,1.3.12.2.1107.5.12.7.5054.30000020050900111600...,3518,2800,NaN,No Box,NaN,BI-RADS 1,DENSITY-C,image,R,MLO,BSMINHBM,"[0, 1, 0, 0, 0, 0, 0, 0, 1, 0]",train,/home/single1/BACKUP/SamHUyen/multi_view_mammo...,"[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,..."
3,2005012602,1.2.410.200010.1064800.1782.1119751.1134541.11...,1.3.12.2.1107.5.12.7.5054.30000020050900111600...,1.3.12.2.1107.5.12.7.5054.30000020050900111600...,3518,2800,NaN,No Box,NaN,BI-RADS 1,DENSITY-C,image,L,MLO,BSMINHBM,"[0, 1, 0, 0, 0, 0, 0, 0, 1, 0]",train,/home/single1/BACKUP/SamHUyen/multi_view_mammo...,"[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,..."
4,1907032728,1.2.410.200010.1064926.5385.626127.635602.635602,1.3.12.2.1107.5.12.7.5054.30000019071600261965...,1.3.12.2.1107.5.12.7.5054.30000019071600261965...,3518,2800,NaN,No Box,NaN,BI-RADS 1,DENSITY-C,NaN,R,CC,NaN,"[0, 1, 0, 0, 0, 0, 0, 0, 1, 0]",train,/home/single1/BACKUP/SamHUyen/multi_view_mammo...,"[[0.0, 1.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 1.0,..."


In [24]:
labels_df[labels_df["fold"]=="holdout"].to_csv("/home/single1/BACKUP/SamHUyen/mammo/sam/singleview_sam/holdoutprdict_model_fullimg.csv")

In [87]:
labels_df[labels_df["fold"]=="holdout"]["multilabel"].unique()

array(['[0, 0, 1, 0, 0, 0, 0, 0, 1, 0]', '[0, 1, 0, 0, 0, 0, 0, 0, 1, 0]',
       '[0, 0, 0, 1, 0, 0, 0, 0, 1, 0]', '[0, 0, 0, 0, 1, 0, 0, 0, 1, 0]',
       '[0, 0, 1, 0, 0, 0, 0, 1, 0, 0]', '[0, 1, 0, 0, 0, 0, 0, 1, 0, 0]',
       '[0, 1, 0, 0, 0, 0, 0, 0, 0, 1]', '[0, 0, 1, 0, 0, 0, 0, 0, 0, 1]',
       '[1, 0, 0, 0, 0, 0, 0, 0, 1, 0]', '[0, 1, 0, 0, 0, 0, 1, 0, 0, 0]',
       '[0, 0, 0, 0, 0, 1, 0, 0, 1, 0]', '[0, 0, 0, 1, 0, 0, 0, 0, 0, 1]',
       '[0, 0, 0, 0, 1, 0, 0, 1, 0, 0]', '[0, 0, 0, 1, 0, 0, 0, 1, 0, 0]',
       '[0, 0, 1, 0, 0, 0, 1, 0, 0, 0]', '[0, 0, 0, 0, 1, 0, 0, 0, 0, 1]',
       '[1, 0, 0, 0, 0, 0, 0, 0, 0, 1]'], dtype=object)

In [14]:
list_truevalue_holduot = []
for i in labels_df[labels_df["fold"]=="holdout"]["multilabel"].values:
    #print(i)
    temp = i.strip('][').split(',')
    #print(temp)
    temp = np.array(temp).astype('int32').tolist()
    list_truevalue_holduot.append(temp)
print(len(list_truevalue_holduot))

2900


In [17]:
list_predict_holdout = []
for i in label_holdout:#labels_df[labels_df["fold"]=="holdout"]["predict"].values:
    list_predict_holdout.append(i.astype('int32')[0].tolist())

In [16]:
list_truevalue_holduot[:3]

[[0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 1, 0]]

In [75]:
def check_0 (Tu, Mau):
    result = 0
    if Mau !=0:
        result = Tu/Mau
    return result


def f1score_mullabel(labels, y_true, y_pred):
    TP = 0
    FP = 0
    FN = 0
    TN = 0
    for ind in range(len(y_true)):
        if y_pred[ind] == labels:
            if  y_true[ind] == y_pred[ind]:
                TP = TP + 1 #predict positive and true value positive
            else:
                FP = FP + 1 #predict positive but true value negative
        else:
            if y_true[ind] == labels:
                FN = FN + 1 #predict negative but true value positive
            else:
                TN = TN + 1 #predict negative and true value negative
    if (TP+ FP) != 0 :
        precision = TP / (TP+ FP)
    else:
        precision = 0
    recall = TP/(TP+FN)
    f1_score= (2*precision*recall)/(precision+recall)
    return TP, FP, FN, TN, f1_score

In [11]:
def acc_score(y_true, y_pred):
    TP = 0
    for ind in range(len(y_true)):
        if y_pred[ind] == y_true[ind]:
            TP += 1
    return TP/2900

In [18]:
acc_score(list_truevalue_holduot, list_predict_holdout)

0.5734482758620689

In [97]:
TP, FP, FN, TN, f1_score = f1score_mullabel([0, 0, 1, 0, 0, 0, 0, 0, 1, 0],list_truevalue_holduot, list_predict_holdout )
print("True Positive: ", TP)
print("False Positive: ", FP)
print("False Negative: ", FN)
print("True Negative: ", TN)
print("F1-score: ", f1_score)



True Positive:  4
False Positive:  1
False Negative:  462
True Negative:  2433
F1-score:  0.016985138004246284


In [95]:
list_truevalue_holduot[:5]

[[0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
 [0, 0, 1, 0, 0, 0, 0, 0, 1, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 1, 0]]

In [91]:
list_predict_holdout[:5]

[[0, 1, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 1, 0],
 [0, 1, 0, 0, 0, 0, 0, 0, 1, 0]]

In [3]:
import torch
torch.sum(torch.tensor([0,1,0,0,0,1]) == torch.tensor([0,1,0,0,0,1]))

tensor(6)